In [4]:
project_folder = "/data_1/ATM/data_1/sfm/projects/EGU2"

parse the gcps xml and convert to pandas dataframe

In [6]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

def parse_gcp_xml(xml_file, type):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Create a list to store the data
    data = []

    if type == "image":
        # Iterate through each 'MesureAppuiFlottant1Im' in the XML
        for measure in root.findall('MesureAppuiFlottant1Im'):
            image_id = measure.find('NameIm').text
            # Iterate through each 'OneMesureAF1I' within the current 'MesureAppuiFlottant1Im'
            for pt in measure.findall('OneMesureAF1I'):
                gcp = pt.find('NamePt').text
                xy = pt.find('PtIm').text.split()
                x = int(xy[0])
                y = int(xy[1])

                # Append the extracted information to the data list
                data.append({'image_id': image_id, 'gcp': gcp, 'x': x, 'y': y})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['image_id', 'gcp', 'x', 'y'])

    elif type == "world":
        # Iterate through each 'OneAppuisDAF' in the XML
        for point in root.findall('OneAppuisDAF'):
            gcp = point.find('NamePt').text
            xy = point.find('Pt').text.split()
            x = float(xy[0])
            y = float(xy[1])
            z = float(xy[2])
            incertitude = point.find('Incertitude').text.split()
            x_quality = float(incertitude[0])
            y_quality = float(incertitude[1])
            z_quality = float(incertitude[2])

            # Append the extracted information to the data list
            data.append({'gcp': gcp, 'x': x, 'y': y, 'z': z,
                         'x_quality': x_quality, 'y_quality': y_quality, 'z_quality': z_quality})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['gcp', 'x', 'y', 'z', 'x_quality', 'y_quality', 'z_quality'])

    else:
        raise ValueError("Invalid type argument. Use 'image' or 'world'.")

    return df

path_gcp_files_image = os.path.join(project_folder, "Measures-S2D.xml")
path_gcp_files_real = os.path.join(project_folder, "Measures.xml")

gcps_image = parse_gcp_xml(path_gcp_files_image, "image")
gcps_real = parse_gcp_xml(path_gcp_files_real, "world")

gcps_image.columns=['image_id', 'gcp', 'x_img', 'y_img']
gcps_real.columns=['gcp', 'x_real', 'y_real', 'z_real', 'x_quality', 'y_quality', 'z_quality']

data = gcps_image.merge(gcps_real, on='gcp', how='inner')

print(data)


                       image_id    gcp  x_img  y_img     x_real    y_real  \
0   OIS-Reech_CA180132V0095.tif   GCP1   8138    688 -1976459.0  767651.5   
1   OIS-Reech_CA180132V0097.tif   GCP1   2648   1310 -1976459.0  767651.5   
2   OIS-Reech_CA180132V0095.tif   GCP2   3423   1569 -1980396.0  768833.0   
3   OIS-Reech_CA180132V0094.tif   GCP2   6313   1559 -1980396.0  768833.0   
4   OIS-Reech_CA180132V0095.tif   GCP3   2988   3306 -1981393.0  767669.0   
5   OIS-Reech_CA180132V0096.tif   GCP3    290   3636 -1981393.0  767669.0   
6   OIS-Reech_CA180132V0095.tif   GCP4    852   4175 -1983357.0  767838.5   
7   OIS-Reech_CA180132V0094.tif   GCP4   3789   4100 -1983357.0  767838.5   
8   OIS-Reech_CA180132V0095.tif   GCP5    954   4417 -1983371.5  767614.5   
9   OIS-Reech_CA180132V0094.tif   GCP5   3881   4333 -1983371.5  767614.5   
10  OIS-Reech_CA180132V0095.tif   GCP6   7174   4886 -1978798.5  764802.0   
11  OIS-Reech_CA180132V0097.tif   GCP6   1776   5682 -1978798.5  764802.0   

In [7]:
import random
import numpy as np

import src.load.load_image as li

import src.display.display_images as di

# get the unique image ids and shuffle
image_ids = data['image_id'].unique()
random.shuffle(image_ids)

for base_image_id in image_ids:

    # load image
    base_image = li.load_image(os.path.join(project_folder, base_image_id))

    # get all rows for the base image
    base_data = data[data['image_id'] == base_image_id]

    # get the ids for base image
    base_cgps = base_data['gcp'].unique()

    # get coordinates of the base gcps
    base_coords = base_data[['gcp', 'x_img', 'y_img']].values

    # get other ids and shuffle
    other_ids = image_ids[image_ids != base_image_id]
    random.shuffle(other_ids)

    for other_image_id in other_ids:

        # get all rows for the other image
        other_data = data[data['image_id'] == other_image_id]

                # get coordinates of the other gcps
        other_coords = other_data[['gcp', 'x_img', 'y_img']].values

        # check if the other image has the same gcps as the base image
        other_cgps = other_data['gcp'].unique()
        common_gcps = list(set(base_cgps) & set(other_cgps))
        if len(common_gcps) == 0:
            continue

        # get the common coords
        base_common_coords = base_data[base_data['gcp'].isin(common_gcps)][['gcp', 'x_img', 'y_img']].values
        other_common_coords = other_data[other_data['gcp'].isin(common_gcps)][['gcp', 'x_img', 'y_img']].values

        # sort the coords by gcps
        base_common_coords = base_common_coords[base_common_coords[:, 0].argsort()]
        other_common_coords = other_common_coords[other_common_coords[:, 0].argsort()]

        # get coordinates as numpy arrays
        base_common_coords = base_common_coords[:, 1:]
        other_common_coords = other_common_coords[:, 1:]

        # concat the base and other coords
        tie_points = np.concatenate((base_common_coords, other_common_coords), axis=1)

        print(tie_points)


        # load image
        other_image = li.load_image(os.path.join(project_folder, other_image_id))

        style_config = {
            "title": f"{base_image_id} : {other_image_id}",
        }
        di.display_images([base_image, other_image], tie_points=tie_points, style_config=style_config)



[[2648 1310 8138 688]
 [1776 5682 7174 4886]]
[[6603 4156 1123 4611]
 [6484 1433 1170 1940]
 [7094 2123 1743 2584]]
[[4578 5854 7222 5359]
 [5078 3125 7768 2741]]
[[2826 5783 5850 5810]
 [3232 6602 6221 6592]]
[[6313 1559 3423 1569]
 [3789 4100 852 4175]
 [3881 4333 954 4417]]


KeyboardInterrupt: 